In [1]:
import mysql.connector 
import config_aws
import requests
import time  # This is just to be able to make multiple calls to the API in spaced out time intervals
import string # For the remove_special_chars function
import pandas as pd
from sodapy import Socrata
import sqlalchemy as db
import pandas as pd
# from helpers import (query, query_list)

In [2]:
from statsmodels.formula.api import ols
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from copy import deepcopy
from math import sqrt, ceil
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                        MultiComparison)
from sklearn.datasets import load_boston

from sqlalchemy import select


import matplotlib.pyplot as plt
sns.set(style='white',palette='vlag')
%matplotlib inline

In [3]:
#instantiate the connection and table

engine = db.create_engine(f'mysql+mysqlconnector://{config_aws.user}:{config_aws.pw}@{config_aws.host}/building_permits')

connection = engine.connect()
metadata=db.MetaData()

In [4]:
def query(q):
    try:
        return pd.read_sql_query(q, engine)
    # Pass when no data is returned    
    except ResourceClosedError:
        pass

def query_list(col, table, distinct = True):
    elts = ['SELECT',
            'DISTINCT' if distinct else '',
            col,
            'FROM',
            table]
    query_str = ' '.join(elts)
    df = query(query_str)
    l = df.iloc[:,0].tolist()
    return l

In [5]:
# raw_data = query('SELECT * FROM  dob_permits')

In [6]:
raw_data = query('SELECT * FROM dob_permits WHERE dob_permits.borough = "MANHATTAN"')

In [7]:
raw_data.head()

,bin__,bldg_type,block,borough,city,community_board,dobrundate,expiration_date,filing_date,filing_status,...,permit_status,permit_type,permittee_s_business_name,permittee_s_license__,permittee_s_license_type,residential,self_cert,work_type,zip_code,id
0,1023743,2,1010,MANHATTAN,NEW YORK,105,2019-12-09,2020-12-08,2019-12-08,RENEWAL,...,ISSUED,EW,None,613862,GC,YES,N,OT,10019,2
1,1057312,2,1893,MANHATTAN,NY,107,2019-12-09,2020-12-08,2019-12-08,RENEWAL,...,ISSUED,EW,None,613862,GC,YES,N,MH,10025,3
2,1057312,2,1893,MANHATTAN,NY,107,2019-12-09,2020-12-08,2019-12-08,RENEWAL,...,ISSUED,EW,None,613862,GC,YES,N,OT,10025,5
3,1035778,2,1292,MANHATTAN,CARLSTADT,105,2019-12-07,2020-12-05,2019-12-06,INITIAL,...,ISSUED,EQ,None,608211,GC,No,Y,EQ,10022,7
4,1083751,2,1053,MANHATTAN,NEW YORK,104,2019-12-07,2020-12-01,2019-12-06,RENEWAL,...,RE-ISSUED,AL,None,619476,GC,YES,N,NA,10036,8


In [8]:
#Make a less intensive test dataset:

# raw_data = raw_data.set_index('borough').loc[['MANHATTAN', 'BROOKLYN']]
# raw_data = raw_data.reset_index()

In [9]:
raw_data.shape

(411098, 37)

Visualization:

In [13]:
#classify owners business type
# 1 = Corp, 1 = IND, 3 = GOV, 

def ob_type_categorize(x):
    if x == "CORPORATION":
        return 1
    if x == "INDIVIDUAL":
        return 2
    if x == "CONDO/CO-OP":
        return 2
    if x == "OTHER GOV'T AGENCY":
        return 3
    if x == "NYCHA/HHC":
        return 3
    if x == "NYC AGENCY":
        return 3
    if x == "NYCHA":
        return 3
    if x == "HPD":
        return 3
    if x == "DOE":
        return 3
    if x == "HHC":
        return 3
    if x == "DCAS":
        return 3
    if x == "NY STATE":
        return 3
    if x == "PARTNERSHIP":
        return 4
    if x == "OTHER":
        return 5
    else:
        return 5

    
# test_dataset['cat_owner_business_type'] = test_dataset['owner_s_business_type'].map(lambda x: ob_type_categorize(x))
# test_dataset['cat_owner_business_type'].value_counts()



Classify work_type:

- BL: Boiler
- CC: Curb Cut
- CH: Chute
- DM: Demolition and Removal
- EQ: Construction Equipment
- EW: Equipment Work
- FA: Fire Alarm
- FB: Fuel Burning
- FN: Fence
- FP: Fire Suppression
- FS: Fuel Storage
- MH: Mechanical/HVAC
- OT: Other
- PL: Plumbing
- SD: Standpipe
- SF: Scaffold
- SG: Sign
- SP: Sprinkler
- EL: Elevator

In [70]:
# Classify work type
def cat_work(x):
    if x == 'OT':
        return 1
    if x == 'PL':
        return 2
    if x == 'EQ':
        return 3
    if x == 'NA':
        return 1
    if x == 'MH':
        return 4     
    if x == 'SP':
        return 5
    if x == 'BL':
        return 5
    if x =='SD':    
        return 5
    if x == 'FP':   #all 6 are fuel related
        return 6
    if x == 'FB': 
        return 6
    if x == 'FS':    
        return 6
    if x == 'CC':
        return 7
    else:
        return 1



In [71]:
# FULL INITIAL CLEAN
#dropping permit status because nearly all are issued
#dropping house is just house number
#dropping 'block', too many, location specific
#dropping 'bin__', DOB id number for specific buildings

def full_clean(data):
    
    #create new column 
    data['time_to_issuance']=data['issuance_date'] - data['filing_date']
    
    #create new column
    data['cat_owner_business_type'] = data['owner_s_business_type'].map(lambda x: ob_type_categorize(x))
    
    data['cat_work_type'] = data['work_type'].map(lambda x: cat_work(x))
    
    #create new column for 
    data['time_to_startdate'] = data['issuance_date'] - data['job_start_date']
    
    #drop columns
    data.drop(columns= ['permittee_s_business_name', 'gis_census_tract', 'dobrundate',
                         'job_start_date', 'city', 'house__', 
                        'permit_status', 'block', 'bin__', 'gis_nta_name', 'expiration_date', 
                        'gis_latitude', 'permittee_s_license__', 'owner_s_business_type',
                        'issuance_date','work_type', 'borough', 'lot', 'permit_si_no', 
                        'owner_s_zip_code', 'gis_longitude'], inplace=True)
    
    data.drop(columns=['time_to_startdate ','filing_date ', 'time_to_issuance'], inplace=True)
    #categorize
    
    return data

In [72]:
df = full_clean(raw_data)

KeyError: "['time_to_startdate ' 'filing_date '] not found in axis"

In [12]:
df.info()

NameError: name 'df' is not defined

**Types of features**:

Dependentent Variable:
- residential (Y, N)

Categorical:
- self_cert (Y, N, NA)
- zip_code
- borough (2)
- community_board (36)
- nonprofit (Y, N) likely correlated
- work_type (12 - categorized) now 'cat_work_type'
- job_type (A2, A3, A1, NB, DM, SG)
- permit_type (EW, PL, EQ, AL, NB, FO, SG, DM) # going to be correlated with job_type!
- permittee_liscense_type (GC, MP, FS, OB, SI, NW, OW, RA, PE)
- cat_owner_business_type  
- gis_nta_name (80 different location names, neighborhoods) DROP???
- owner_s_zip_code (2305 total values)
- permit_sequence__ (29 options, number of a sequence type)
- gis_council_district (26 options)
- gis_census_tract (861 options)

Continuous:
- dobrundate (DROP???)
- time_to_issuance
- time_to_startdate 

Ordinal:

Not sure:
- permittee_liscense 15682 unique, maybe id number
-